In [1]:
import pandas as pd

#### Get the index

In [15]:
results_per_page = 1000
query = "Dollar%20General"
url = f"https://www.osha.gov/ords/imis/establishment.search?establishment={query}&state=all&officetype=all&office=all&sitezip=100000&startmonth=11&startday=18&startyear=2012&endmonth=11&endday=18&endyear=2022&p_case=all&p_sort=12&p_desc=DESC&p_direction=Prev&p_show={results_per_page}&p_violations_exist=yes"

df = pd.read_html(url)[2]

In [27]:
df

,Unnamed: 0,#,Activity,Opened,RID,St,Type,Sc,SIC,NAICS,Vio,Establishment Name
0,NaN,1,1.617010e+06,08/25/2022,552652,MI,Complaint,Partial,NaN,452319,1,Dollar General #17771
1,NaN,2,1.607451e+06,07/13/2022,418100,GA,Complaint,Partial,NaN,445110,1,Dollar General Atlanta Fresh Distribution Center
2,NaN,3,1.599901e+06,06/03/2022,418600,FL,Complaint,Partial,NaN,452319,3,"Dollar General Corporation/ Dolgencorp, Llc"
3,NaN,4,1.598544e+06,05/24/2022,418200,GA,Complaint,Partial,NaN,452319,7,"Dollar General Corporation/ Dolgencorp, Llc"
4,NaN,5,1.597065e+06,05/19/2022,418600,AL,Complaint,Partial,NaN,452319,3,"Dollar General Corporation/ Dolgencorp, Llc"
...,...,...,...,...,...,...,...,...,...,...,...,...
126,NaN,127,9.053960e+05,05/06/2013,627700,OK,Planned,Complete,NaN,493110,4,Dollar General Corporation
127,NaN,128,8.928720e+05,03/05/2013,522500,OH,Complaint,Partial,NaN,452112,1,Dollar General #09587
128,NaN,129,8.926460e+05,02/27/2013,627100,AR,Complaint,Partial,NaN,452990,4,Dollar General Store #7495
129,NaN,130,3.166243e+08,02/13/2013,454510,SC,Complaint,Partial,5331.0,452990,1,Dollar General Corporation


In [38]:
for index, row in df.iterrows():
    activity_code = row['Activity']
    url = f"https://www.osha.gov/ords/imis/establishment.inspection_detail?id={activity_code}"
    print(url)

https://www.osha.gov/ords/imis/establishment.inspection_detail?id=1617010.015
https://www.osha.gov/ords/imis/establishment.inspection_detail?id=1607451.015
https://www.osha.gov/ords/imis/establishment.inspection_detail?id=1599901.015
https://www.osha.gov/ords/imis/establishment.inspection_detail?id=1598544.015
https://www.osha.gov/ords/imis/establishment.inspection_detail?id=1597065.015
https://www.osha.gov/ords/imis/establishment.inspection_detail?id=1592340.015
https://www.osha.gov/ords/imis/establishment.inspection_detail?id=1590571.015
https://www.osha.gov/ords/imis/establishment.inspection_detail?id=1590827.015
https://www.osha.gov/ords/imis/establishment.inspection_detail?id=1590051.015
https://www.osha.gov/ords/imis/establishment.inspection_detail?id=1589448.015
https://www.osha.gov/ords/imis/establishment.inspection_detail?id=1588719.015
https://www.osha.gov/ords/imis/establishment.inspection_detail?id=1586442.015
https://www.osha.gov/ords/imis/establishment.inspection_detail?i

In [41]:
# Figure out how to scrape individual activity

url = "https://www.osha.gov/ords/imis/establishment.inspection_detail?id=1617010.015"
tables = pd.read_html(url)
tables[0]

,Type,Activity Nr,Safety,Health
0,Complaint,1868426,Yes,NaN


In [42]:
tables[1]

,Violations/Penalties,Serious,Willful,Repeat,Other,Unclass,Total
0,Initial Violations,NaN,NaN,NaN,1,NaN,1
1,Current Violations,NaN,NaN,NaN,1,NaN,1
2,Initial Penalty,$0,$0,$0,$0,$0,$0
3,Current Penalty,$0,$0,$0,$0,$0,$0
4,FTA Penalty,$0,$0,$0,$0,$0,$0


In [43]:
tables[2]

,#,Citation ID,Citaton Type,Standard,Issuance Date,Abatement Due Date,Current Penalty,Initial Penalty,FTA Penalty,Contest,Latest Event,Note
0,1.0,01001A,Other,19100141 B01 I,09/21/2022,NaN,$0,$0,$0,NaN,Z - Issued,NaN
1,2.0,01001B,Other,19100141 C01 I,09/21/2022,NaN,$0,$0,$0,NaN,Z - Issued,NaN


In [ ]:
# How to get Case Status
# scrape this with beautifulsoup
# search for CSS selector .well

In [ ]:
# How to get Mailing Address
# scrape this with beautifulsoup
# search for CSS selector .well